In [1]:
import pandas as pd
from MCForecastTools import MCSimulation
import requests
from dotenv import load_dotenv
import os
import numpy as np
import seaborn as sns
import yfinance as yf
import panel as pn
from panel.interact import interact
from panel import widgets
import plotly.express as px
pn.extension('plotly')

from scipy import optimize
import datetime
import hvplot.pandas
%matplotlib inline

In [2]:
def return_on_lending(amount, interest_rate, duration):
    def xnpv(rate, cashflows):
        return sum([cf/(1+rate)**((t-cashflows[0][0]).days/365.0) for (t,cf) in cashflows])
 
    def xirr(cashflows, guess=0.1):
        try:
            return optimize.newton(lambda r: xnpv(r,cashflows),guess)
        except:
            print('Calc Wrong')

    loan_amount = amount
    rate = interest_rate
    initial_investment = -loan_amount
    monthly_Payment = loan_amount * rate * (30/360)
    last_payment = loan_amount + monthly_Payment
    date_list =['2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11', '2021-01-12', '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06']
    ids_list = [1,2,3,4,5,6,7,8,9,10,11,12,13]
    flows_list = [ initial_investment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment, monthly_Payment,  monthly_Payment, monthly_Payment, last_payment]
    df = pd.DataFrame(list(zip(date_list,ids_list,flows_list)), columns=['Date','ID','Flow'])
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    data = list(zip(df['Date'].values,df['Flow'].values))
    value = xirr(data)
    return round((value * amount * duration),2)

In [3]:
# GET TICKERS
tickers = input('what do you want to invest in?').split()

what do you want to invest in? AAPL TWTR AMZN BTC-USD ETH-USD


In [4]:
# GET CAPITAL AMOUNT TO BE INVESTED
capital = int(input('how much do you have to invest?'))

how much do you have to invest? 100000


In [5]:
# GET YEARLY DURATION FOR FORECASTING
duration = int(input('in years, chose a duration for the forecasting'))

in years, chose a duration for the forecasting 5


In [6]:
# GET HISTORIAL DATA FROM YAHOO FINANCE FOR ASSETS CHOSEN
df_portfolio = yf.download(tickers, start='2019-03-20', end='2020-03-20')

[*********************100%***********************]  5 of 5 completed


In [7]:
# CREATE DATAFRAME OF JUST CLOSING PRICES
df_closing = df_portfolio['Adj Close'].dropna()

In [8]:
# CALCULATE DAILY RETURNS
daily_returns = df_closing.pct_change().dropna()

In [9]:
# CALCULATE CORRELATION OF ASSETS 
correlations = daily_returns.corr()

In [10]:
# CALCULATE ANNUALIZED STANDARD DEVIATION OF ASSETS
std_deviations = daily_returns.std()
annualized_std = (std_deviations * np.sqrt(252)).sort_values(ascending=False)

In [11]:
# CALCULATE SHARPE RATIOS
sharpe_ratios = ((daily_returns.mean() * 252)) / (daily_returns.std() * np.sqrt(252))

In [12]:
# CREATE WEIGHTS FOR PORTFOLIOS
low_risk_weights = [.05, .1, .15, .2, .5]
med_risk_weights = [.1, .15, .2, .25, .3]
high_risk_weights = [.3, .25, .2, .15, .1]

# LIST OF ASSETS IN ORDER OF MOST VOLATILE TO LEAST VOLATILE 
volatitly_ranks = annualized_std.index.to_list()

# DAILY RETURNS DATAFRAME REORDERED IN TERMS OF VOLATILITY
ordered_volatility_df_pct = pd.concat([daily_returns[volatitly_ranks[0]], daily_returns[volatitly_ranks[1]],daily_returns[volatitly_ranks[2]],daily_returns[volatitly_ranks[3]],daily_returns[volatitly_ranks[4]]], axis=1, join='inner')

#CLOSING PRICES DATAFRAME REORDERED IN TERMS OF VOLATILITY
ordered_volatility_df_closing = pd.concat([df_closing[volatitly_ranks[0]], df_closing[volatitly_ranks[1]],df_closing[volatitly_ranks[2]],df_closing[volatitly_ranks[3]],df_closing[volatitly_ranks[4]]], axis=1, join='inner')

# RISK APPETITE PORTOLIOS USING DAILY RETURNS DATA
low_risk_portfolio_pct = ordered_volatility_df_pct.dot(low_risk_weights)
med_risk_portfolio_pct = ordered_volatility_df_pct.dot(med_risk_weights)
high_risk_portfolio_pct = ordered_volatility_df_pct.dot(high_risk_weights)

# RISK APPETITE PORTOLIOS USING CLOSE PRICES DATA
low_risk_portfolio_close = ordered_volatility_df_closing.dot(low_risk_weights)
med_risk_portfolio_close = ordered_volatility_df_closing.dot(med_risk_weights)
high_risk_portfolio_close = ordered_volatility_df_closing.dot(high_risk_weights)

In [13]:
# CALCULATE CUMULATIVE RETURNS FOR ALL PORTFOLIOS (DAILY RETURNS DATA)
low_risk_cumulative_returns = (1 + low_risk_portfolio_pct).cumprod() - 1
med_risk_cumulative_returns = (1 + med_risk_portfolio_pct).cumprod() - 1
high_risk_cumulative_returns = (1 + high_risk_portfolio_pct).cumprod() - 1

In [14]:
# CREATE MULTI LEVEL DATAFRAME FOR MONTE CARLO SIMULATIONS
multi_level = pd.concat([ordered_volatility_df_closing], axis=1, keys=['close']).swaplevel(0,1,1)

In [15]:
# CREATE MC INSTANCES OF LOW, MED AND HIGH WEIGHTS

MC_low_risk = MCSimulation(
    portfolio_data = multi_level,
    weights = low_risk_weights,
    num_simulation = 100,
    num_trading_days = 252*duration
)

MC_med_risk = MCSimulation(
    portfolio_data = multi_level,
    weights = med_risk_weights,
    num_simulation = 100,
    num_trading_days = 252*duration
)

MC_high_risk = MCSimulation(
    portfolio_data = multi_level,
    weights = high_risk_weights,
    num_simulation = 100,
    num_trading_days = 252*duration
)

In [16]:
# RUN MONTE CARLO CUMULATIVE RETURN FOR LOW RISK
MC_low_risk.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.007751,0.985897,1.000787,0.981394,1.006785,0.994025,1.001309,0.992919,1.020515,1.026007,...,0.999466,0.984962,0.996991,1.009080,1.016911,0.995070,1.016389,0.997300,1.006473,0.989011
2,1.018655,1.001261,0.986419,0.988236,1.014251,1.000532,0.999177,1.013775,1.031203,1.004220,...,1.015042,0.996387,1.005336,0.995392,1.027330,1.002943,1.052318,0.963191,1.000313,0.981348
3,1.016133,0.991796,0.976252,0.990726,0.995181,0.989629,1.005547,1.029372,1.027785,1.014065,...,1.022701,1.005606,1.018514,0.998105,1.033438,1.031336,1.047232,0.958669,1.000263,1.018204
4,1.018291,1.003186,0.953598,0.998744,1.006781,0.993290,0.999607,1.021528,1.016937,1.018532,...,0.999499,0.989055,1.032283,1.009567,1.030827,1.040674,1.040247,0.966588,0.986334,1.006971
5,1.020014,1.019067,0.955133,1.010391,0.992259,0.985044,1.002541,1.032953,1.014827,1.016306,...,1.012597,0.999281,1.038053,1.012121,1.015035,1.025040,1.045708,0.978205,0.989537,0.998957
6,1.016020,1.011455,0.954484,1.012873,0.990017,0.987552,1.032272,1.029856,1.010256,1.013298,...,1.028150,0.988144,1.028688,1.001157,1.002855,1.032540,1.031806,0.967627,0.979570,0.981917
7,1.017363,1.006849,0.971310,1.015258,0.983562,0.987589,1.026142,1.023129,1.032507,1.001518,...,1.038058,0.983064,1.022197,0.991510,1.020256,1.053092,1.022882,0.991961,0.970720,0.981573
8,1.022208,1.021123,0.979472,0.998982,0.972201,0.973874,1.012616,1.022619,1.015418,0.999846,...,1.033713,0.993549,1.032084,1.016652,1.012248,1.035257,1.029873,1.016940,0.964033,0.973572
9,1.005692,1.031063,0.982040,0.989985,0.968288,0.985965,1.043310,1.037929,1.024419,1.000078,...,1.019100,0.989530,1.071136,1.013465,1.021494,1.013558,1.046265,1.003823,0.966553,0.989128


In [17]:
# RUN MONTE CARLO CUMULATIVE RETURN FOR MEDIUM RISK
MC_med_risk.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.012811,0.989065,1.002237,1.002698,0.993354,1.000434,0.986670,1.014654,1.007764,1.011299,...,1.013278,1.008540,0.982935,1.010478,0.973979,0.991808,1.003673,1.005780,0.984834,1.012397
2,1.011940,0.993001,0.987236,1.023875,0.966077,1.010048,0.977305,1.016650,1.013740,1.008013,...,1.016255,0.995426,0.974153,1.001347,0.976930,0.978308,0.981668,1.022754,0.996661,1.029586
3,1.009117,0.972393,0.997874,1.019132,0.954979,1.010785,0.972301,1.024491,0.992064,1.019865,...,1.029815,0.978824,0.965422,0.991113,0.993901,0.986331,0.964361,1.002539,0.996506,1.005425
4,1.003026,0.965063,1.012646,1.043176,0.967691,0.981107,0.974378,1.057100,0.980368,0.999109,...,1.034864,1.003546,0.994391,0.976052,0.986111,0.966936,0.979516,0.989622,0.997083,1.006995
5,0.996387,0.959668,1.022334,1.036660,0.980681,0.970210,0.934774,1.044088,0.982444,1.003835,...,1.021074,1.008647,0.973759,0.970122,0.982775,0.965655,0.972445,0.979921,0.990280,1.023660
6,1.018109,0.957249,1.013991,1.041114,0.983156,0.962216,0.955485,1.037727,0.993659,1.006820,...,1.035436,1.009312,0.996262,0.947105,1.000582,0.966300,0.965786,1.002130,0.987496,1.030043
7,1.003014,0.966345,0.986605,1.033595,0.999514,0.988641,0.954888,1.041187,0.995196,1.027943,...,1.031709,1.027315,1.009716,0.947563,1.009954,0.962624,0.946452,0.997446,0.981853,1.040399
8,1.005984,0.966435,0.992663,1.032813,1.006480,0.965884,0.960088,1.070300,0.984106,1.051330,...,1.014876,1.004588,0.998355,0.953605,1.011276,0.964874,0.974302,0.989331,1.005440,1.063561
9,0.990879,0.965455,1.013495,1.030390,1.010658,0.966735,0.958715,1.066627,0.971858,1.048462,...,1.017086,0.985692,1.002200,0.940769,0.996516,0.976680,0.950349,0.983435,0.995731,1.065462


In [18]:
# RUN MONTE CARLO CUMULATIVE RETURN FOR HIGH RISK
MC_high_risk.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.012839,1.014724,0.963098,0.999833,0.942978,0.994607,0.981828,1.000662,1.054238,1.021815,...,1.004598,1.007181,0.982697,1.037597,0.987356,0.981683,0.992493,1.040331,0.986443,1.024942
2,0.991798,1.018382,0.982413,0.979180,0.935967,1.003688,0.985152,0.991142,1.072197,1.032818,...,0.990892,0.995500,0.995775,1.032441,0.956954,0.981339,1.014537,1.043074,0.966863,0.982877
3,0.974062,1.034117,0.969740,0.965443,0.952301,1.011077,0.960914,1.023371,1.095423,1.033823,...,0.975259,0.981130,1.043459,1.046329,0.980469,0.953194,1.031968,1.031259,0.994889,0.988109
4,0.989786,1.035192,1.008990,0.978631,0.958824,1.031573,0.943804,1.024911,1.106085,0.997229,...,0.963202,0.983403,1.082898,1.047035,0.992121,0.974668,1.050345,1.083817,1.009012,1.001252
5,0.979293,0.997943,0.994428,0.998217,0.936254,1.048242,0.955191,1.029455,1.124006,0.971982,...,0.990169,0.964003,1.121722,1.006478,1.001866,1.001366,1.048221,1.098048,1.018475,0.998988
6,1.032871,1.006748,1.014364,0.987727,0.936045,1.043358,0.946659,1.017003,1.087252,0.928917,...,0.972271,0.970167,1.099005,0.991110,1.022625,1.039499,1.043933,1.125791,1.016277,1.026464
7,1.038808,1.024283,1.024283,1.026275,0.985367,1.022132,0.947482,1.036728,1.101034,0.980772,...,0.970208,0.932123,1.105159,1.017487,1.028953,1.051151,1.017945,1.126454,1.004110,1.020219
8,1.043773,1.021471,1.014540,1.010726,0.982374,1.031423,0.936462,1.040188,1.072459,0.991259,...,0.979981,0.956834,1.157756,1.030316,1.011332,1.014398,1.058883,1.123578,1.010903,1.053486
9,1.048104,0.994505,1.020578,1.031026,0.956853,1.021596,0.955504,1.052633,1.056568,0.974486,...,0.986296,0.971604,1.142233,1.013845,1.051626,0.992029,1.085925,1.146799,1.003823,1.070747


In [19]:
# CREATE SUMMARY STATS FOR EACH MONTE CARLO SIMULATION
low_risk_table = MC_low_risk.summarize_cumulative_return()
med_risk_table = MC_med_risk.summarize_cumulative_return()
high_risk_table = MC_high_risk.summarize_cumulative_return()

In [24]:
# RISK/ASSET ANALYSIS VISUALIZATIONS
from bokeh.models import NumeralTickFormatter

daily_returns_hist = daily_returns.hvplot.hist(ylabel="Frequency", xlabel="Percent Change",bins=100, figsize=(25,10))
closing_prices_line = df_closing.hvplot.line(ylabel="Price", xlabel="Date", frame_width=1000, frame_height=400, logy=True).opts(yformatter=NumeralTickFormatter(format="$0,000.00"))
correlations_heatmap = correlations.hvplot.heatmap(cmap=["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"])
std_devs_bar = annualized_std.hvplot.bar(ylabel="Standard Deviation")
sharpe_ratios_bar = sharpe_ratios.hvplot.bar(ylabel="Sharpe Ratio")

# PORTFOLIO PERFORMANCE VISUALIZATIONS
low_risk_close_plot = low_risk_portfolio_close.hvplot.line(ylabel="Price",label="Low Risk").opts(show_legend=True)
med_risk_close_plot = med_risk_portfolio_close.hvplot.line(label="Medium Risk").opts(show_legend=True)
high_risk_close_plot = high_risk_portfolio_close.hvplot.line(label="High Risk").opts(show_legend=True)
portfolios_plot_close_data = (low_risk_close_plot * med_risk_close_plot * high_risk_close_plot).opts(ylabel="Price",
    legend_position="top_left",yformatter=NumeralTickFormatter(format="$0,000.00"))

low_risk_cumulative_plot = low_risk_cumulative_returns.hvplot(ylabel="Percentage Return",label="Low Risk").opts(show_legend=True)
med_risk_cumulative_plot = med_risk_cumulative_returns.hvplot(label="Medium Risk").opts(show_legend=True)
high_risk_cumulative_plot = high_risk_cumulative_returns.hvplot(label="High Risk").opts(show_legend=True)
portfolios_plot_cumulative_data = (low_risk_cumulative_plot * med_risk_cumulative_plot * high_risk_cumulative_plot).opts(legend_position="top_left",shared_axes=False)

# MONTE CARLO VISUALIZATIONS
low_returns_data = {
    "mean": list(MC_low_risk.simulated_return.mean(axis=1)),
    "median":list(MC_low_risk.simulated_return.median(axis=1)),
    "max":list(MC_low_risk.simulated_return.max(axis=1)),
    "min":list(MC_low_risk.simulated_return.min(axis=1))
}
low_return_df = pd.DataFrame(low_returns_data)
low_risk_MC_plot = low_return_df.hvplot(ylabel="Simulated Cumulative Returns",xlabel="Number of Trading Days")

medium_returns_data = {
    "mean": list(MC_med_risk.simulated_return.mean(axis=1)),
    "median":list(MC_med_risk.simulated_return.median(axis=1)),
    "max":list(MC_med_risk.simulated_return.max(axis=1)),
    "min":list(MC_med_risk.simulated_return.min(axis=1))
}
medium_return_df = pd.DataFrame(medium_returns_data)
med_risk_MC_plot = medium_return_df.hvplot(ylabel="Simulated Cumulative Returns",xlabel="Number of Trading Days")

high_returns_data = {
    "mean": list(MC_high_risk.simulated_return.mean(axis=1)),
    "median":list(MC_high_risk.simulated_return.median(axis=1)),
    "max":list(MC_high_risk.simulated_return.max(axis=1)),
    "min":list(MC_high_risk.simulated_return.min(axis=1))
}
high_return_df = pd.DataFrame(high_returns_data)
high_risk_MC_plot = high_return_df.hvplot(ylabel="Simulated Cumulative Returns",xlabel="Number of Trading Days")



In [25]:
def lendingViz():
    return interact(return_on_lending, amount=(capital*.2,capital*.5), interest_rate=(0.01,0.07,0.001), duration=duration)

In [27]:
# DASHBOARD CREATION

daily_returns_col = pn.Column('## Daily Returns', daily_returns_hist)
sharpe_ratios_col = pn.Column('## Sharpe Ratios', sharpe_ratios_bar)
daily_closing_col = pn.Column('## Closing Prices', closing_prices_line)
std_dev_col = pn.Column('## Annualized STDs', std_devs_bar)
correlations_col = pn.Column('## Correlations', correlations_heatmap)
low_risk_mc_col = pn.Column('## Monte Carlo Low Risk Portfolio', low_risk_MC_plot)
med_risk_mc_col = pn.Column('## Monte Carlo Medium Risk Portfolio', med_risk_MC_plot)
high_risk_mc_col = pn.Column('## Monte Carlo High Risk Portfolio', high_risk_MC_plot)

portfolio_close_col = pn.Column('## Portfolio Close Data', portfolios_plot_close_data)
portfolio_cumulative_col = pn.Column('## Portfolio Cumulative Returns', portfolios_plot_cumulative_data)

risk_analysis_tabs = pn.Tabs(
    ('Closing Prices', daily_closing_col),
    ('Daily Returns', daily_returns_col),
    ('Sharpe Ratios', sharpe_ratios_col),
    ('Standard Deviations', std_dev_col),
    ('Correlations', correlations_col)
)

lending_tab = pn.Tabs(
    ('Lending Interaction', lendingViz())
)

portfolio_performace_tab = pn.Tabs(
    ('Daily Closes', portfolio_close_col),
    ('Cumulative Returns', portfolio_cumulative_col)
)

mc_tab = pn.Tabs(
    ('Low', low_risk_mc_col),
    ('Medium', med_risk_mc_col),
    ('High', high_risk_mc_col)
)
mainTabs = pn.Tabs(
    ('Risk Analysis', risk_analysis_tabs),
    ('Performance', portfolio_performace_tab),
    ('Monte Carlo', mc_tab),
    ('Lending', lending_tab)
)
dashboard = pn.Column('# Analysis', mainTabs)

In [28]:
dashboard

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Tabs
            [0] Column
                [0] Markdown(str)
                [1] HoloViews(NdOverlay)
            [1] Column
                [0] Markdown(str)
                [1] HoloViews(NdOverlay)
            [2] Column
                [0] Markdown(str)
                [1] HoloViews(Bars)
            [3] Column
                [0] Markdown(str)
                [1] HoloViews(Bars)
            [4] Column
                [0] Markdown(str)
                [1] HoloViews(HeatMap)
        [1] Tabs
            [0] Column
                [0] Markdown(str)
                [1] HoloViews(Overlay)
            [1] Column
                [0] Markdown(str)
                [1] HoloViews(Overlay)
        [2] Tabs
            [0] Column
                [0] Markdown(str)
                [1] HoloViews(NdOverlay)
            [1] Column
                [0] Markdown(str)
                [1] HoloViews(NdOverlay)
            [2] Column
                [0] Markdown(str)
                [1] HoloViews(NdOverlay)
        [3] Tabs
            [0] Column
                [0] Column
                    [0] FloatSlider(end=50000.0, name='amount', start=20000.0, value=35000.0, value_throttled=35000.0)
                    [1] FloatSlider(end=0.07, name='interest_rate', start=0.01, step=0.001, value=0.04, value_throttled=0.04)
                    [2] IntSlider(end=15, name='duration', start=-5, value=5, value_throttled=5)
                [1] Row
                    [0] Str(float64, name='interactive05749')

In [29]:
 # Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments in stocks
def mc_and_lending_summary(capital, duration):
    
    low_risk_investment_amount = capital * .8
    low_risk_lending_amount = capital * .2
    
    med_risk_investment_amount = capital * .6
    med_risk_lending_amount = capital * .4
    
    high_risk_investment_amount = capital * .5
    high_risk_lending_amount = capital * .5
    
    low_risk_ci_lower = round(low_risk_table[8]*low_risk_investment_amount,2)
    low_risk_ci_upper = round(low_risk_table[9]*low_risk_investment_amount,2)
    med_risk_ci_lower = round(med_risk_table[8]*med_risk_investment_amount,2)
    med_risk_ci_upper = round(med_risk_table[9]*med_risk_investment_amount,2)
    high_risk_ci_lower = round(high_risk_table[8]*high_risk_investment_amount,2)
    high_risk_ci_upper = round(high_risk_table[9]*high_risk_investment_amount,2)
    
    low_risk_lending_profit = return_on_lending(low_risk_lending_amount, .075, duration)
    med_risk_lending_profit = return_on_lending(med_risk_lending_amount, .1, duration)
    high_risk_lending_profit = return_on_lending(high_risk_lending_amount, .14, duration)
    # Print results
    print(
          f"There is a 95% chance that an initial investment of ${low_risk_investment_amount} in the low risk portfolio"
          f" over the next 5 years will end within in the range of"
          f" ${low_risk_ci_lower} and ${low_risk_ci_upper}. "
          f"Additionally, lending ${low_risk_lending_amount} would yield ${low_risk_lending_profit} in profit." 
         )
    print(
          f"There is a 95% chance that an initial investment of ${med_risk_investment_amount} in the medium risk portfolio"
          f" over the next 5 years will end within in the range of"
          f" ${med_risk_ci_lower} and ${med_risk_ci_upper}. "
          f"Additionally, lending ${med_risk_lending_amount} would yield ${med_risk_lending_profit} in profit."
         )

    print(
          f"There is a 95% chance that an initial investment of ${high_risk_investment_amount} in the high risk portfolio"
          f" over the next 5 years will end within in the range of"
          f" ${high_risk_ci_lower} and ${high_risk_ci_upper}. "
          f"Additionally, lending ${high_risk_lending_amount} would yield ${high_risk_lending_profit} in profit." 
         )

In [30]:
mc_and_lending_summary(capital, duration)

There is a 95% chance that an initial investment of $80000.0 in the low risk portfolio over the next 5 years will end within in the range of $72713.21 and $408763.84. Additionally, lending $20000.0 would yield $7791.19 in profit.
There is a 95% chance that an initial investment of $60000.0 in the medium risk portfolio over the next 5 years will end within in the range of $75181.21 and $479058.34. Additionally, lending $40000.0 would yield $21048.69 in profit.
There is a 95% chance that an initial investment of $50000.0 in the high risk portfolio over the next 5 years will end within in the range of $44499.28 and $868196.26. Additionally, lending $50000.0 would yield $37623.41 in profit.
